# Softmax Layer

**Softmax**

```
f(x) = e^z / sum(e^z)

    - z = alpha * A * B + beta * C
```

**Inputs**

```
- dim: 21
- outer_num_: 300
- inner_num_: 1
- channels: 21
```


**Steps**

for each rois:

- initializing scale_data (outer_num_, inner_num_): (300, 1)
- subtraction (gemm)
- exponentiation (exp)
- sum after exp (gemv)
- division (div)

```
for outer_num_: (i)


    scale_data: initialize
      for channels: (j)
      for inner_num_: (k)
        scale_data[k] = max(scale_data[k], bottom_data[i * dim + j * inner_num_ + k])
        
        
    subtraction: gemm
       C := alpha * A * B + beta * C
       top_data = - sum_multiplier_ * scale_data + C
       
       - M: channels (21)
       - N: inner_num_ (1)
       - K: 1
       - alpha: -1
       - A: sum_multiplier_.cpu_data()
       - lda: 1
       - B: scale_data
       - ldb:
       - beta: 1
       - C: top_data
       - ldc:
       

    exponentiation:
        # top_data = e^top_data
        caffe_exp<Dtype>(dim, top_data, top_data)
        - dim: 21
    
    sum after exp: gemv
        C := alpha * A * x + beta * C
        scale_data = top_data * sum_multiplier_
        
        - CblasRowMajor:
        - TransA: TransA
        - M: channels
        - N: inner_num_
        - alpha: 1
        - A: top_data
        - N: 
        - x: sum_multiplier_.cpu_data()
        - 1
        - beta: 0
        - y: scale_data
        - 1
    
    division:
        for channels:
            # top_data = top_data / scale_data
            caffe_div(inner_num_, top_data, scale_data, top_data);
            top_data += inner_num_;
```


In [1]:
# f(x) = e^z / sum(e^z)
# - z = alpha * A * B + beta * C
#   - A: sum_multiplier
#   - B: scale_data <- generation from bottom_data

In [ ]:
# data
# - scale_data: (300, 1)
# - bottom_data: (300, 21) = 6300
# - top_data: (300, 21) = 6300

# params
# - channels: 21
# - dim: 21
# - outer_num_: 300
# - inner_num_: 1

# steps
#
# - 1. initialize the scale_data
#   - caffe_copy(inner_num_, bottom_data + i * dim, scale_data)
#   - (scale_data) inner_num_: 1
#   - (scale_data) value: bottom_data + [1..300] * 21
#   - intiailize scale_data by copying [bottom_data + i * dim] with sizeof(inner_num_)
#
# - 2. find out the max value of each class per roi
# - scale_data: [300, 1], value: max class score per roi
# - for channels
# - for inner_num_
#   - scale_data[k] = std::max(scale_data[k], bottom_data[i * dim + j * inner_num_ + k]);
#     - i: outer_num_ (300)
#     - j: channels (21)
#     - k: inner_num_ (1)
#     - dim: 21
#     - index: n_rois_i * n_classes + channel_j * inner_num_ + inner_num_k
#   - scale_data[0] = max(scale_data[0], bottom_data[(1..300) * 21 + (1..21) * 1 + 0])
#
# - 3. gemm
# - top_data = - sum_multiplier_ * scale_data + top_data
#   - M: 21
#   - N: 1
#   - K: 1
#   - alpha: -1
#   - beta: 1
#   - lda: 1
#   - ldb: 1
#   - top_data[21, 1] = -sum_multiplier_[21, 1] * scale_data[1, 1] + top_data[21, 1]
#
# - 4. exp
# - top_data [300, 21] = e^(top_data[300, 21])
#
# - 5. sum after exp
# - scale_data = top_data * sum_multiplier_
#
# - 6. division
# - top_data = top_data (4.exp) / scale_data (5.sum_after_exp)